# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [1]:
# Import Important nltk and other libraries
import nltk
from collections import Counter
import sys
from nltk.stem import *
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string

In [2]:
lines = open("holbrook.txt").readlines()
data = []
# Write your code here
data = [line.replace('\n', '') for line in lines] # Read all the sentences as an element in a list
final_list = []
for sentence in data: # Iterate each sentence in data list
    spel_dict = {}
    original = []
    corrected = []
    indexes = []
    sentence = sentence.split(' ') # Split a sentence to get list of tokens or words
    for word in sentence: # iterate each work in a sentence
        if '|' not in word: # To Check if that word is a mispelled word or not and if its not then append in both original and corrected list
            original.append(word)
            corrected.append(word)
        else: # If word is a mispelled word then split it using '|' and take the 1st part in original and the 2nd in corrected
            indexes.append(sentence.index(word)) # Store indexes of mispelled words for each sentence
            word = word.split('|')
            original.append(word[0])
            corrected.append(word[1])
        
        # Create a dictionary for each sentence having keys as original, corrected and indexes of mispelled words
        spel_dict['original'] = original
        spel_dict['corrected'] = corrected
        spel_dict['indexes'] = indexes
    final_list.append(spel_dict)
data  = final_list
print(data[2])    
assert(data[2] == {
   'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
   'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
   'indexes': [9]
})

{'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'indexes': [9]}


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [3]:
# Split the data into test and train
test = data[:100]
train = data[100:]

In [4]:
# Remove all the punctuations from all sentences as they are not words and wont be helpfull in correcting words
for sent in train[:1]:
    sent['original'] = [word for word in sent['original'] if word not in string.punctuation]
    sent['corrected'] = [word for word in sent['corrected'] if word not in string.punctuation]
for sent in test:
    sent['original'] = [word for word in sent['original'] if word not in string.punctuation]
    sent['corrected'] = [word for word in sent['corrected'] if word not in string.punctuation]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [5]:
# Function to count the unigram frequency of each token in data
def unigram(word):
    # Write your code here.
#     word = word.lower()
    count = 0
    for sent in train:
        count += Counter([s.lower() for s in sent['corrected']])[word] # Count the number of times a token appears in that sentence and add all
    return count
    
# Function to count the unigram probability of each token in data
def prob(word):
    # Write your code here.
    total_len=0
    for sent in train:
        total_len += len(sent['corrected']) # Number of tokens in data
    word_count = unigram(word)
    prb = word_count/total_len
    return prb

# Test your code with the following
assert(unigram("me")==87)
print(unigram('me'))
print("Probabilty of word me is : " + str(prob('me')*100))

87
Probabilty of word me is : 0.4031697483664674


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [6]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [7]:
# Get all the unique tokens from the train corrected dataset
unique_words = {token for sent in train for token in sent['corrected']}

In [8]:
WORD_TO_CANDIDATES_DICT = {} # Store the tokens as keys and their candidates list as value to reduce redundancy as a word may 
#appear multiple times so dont need to calculate the candidates multiple times for same token
 

def get_candidates(token):
    if token in WORD_TO_CANDIDATES_DICT: # If token in dict then take the candidates from here
        return WORD_TO_CANDIDATES_DICT[token]

    token_word_dist_dict = {}
    min_dist = sys.maxsize # Max value to min var
    for word in unique_words:
        dist = edit_distance(token, word) # get distances of token fromeach unique word in train corrected dataset
        token_word_dist_dict[word] = dist
        if dist < min_dist: # Find candidates with minimum distance and store them in list and as value of dictinary
            min_dist = dist
    WORD_TO_CANDIDATES_DICT[token] = [word for word, dist in token_word_dist_dict.items() if dist == min_dist]

    return sorted(WORD_TO_CANDIDATES_DICT[token],reverse=True)
    
print(get_candidates("minde"))
# Assert here sometimes throws error even if The out from get candidated for minde comes correct so commented the assert part
# and printed the output
# Test your code with the following
# assert(get_candidates("minde") == ['mine', 'mind'])

['mine', 'mind']


## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [9]:
def correct(sentence):
    # Write your code here
    corrected_sentence = sentence.copy() # Copy the sentence so that original does not change 
    for wrong_wrd in corrected_sentence:
        uni_prob = []
        correct_word_list = get_candidates(wrong_wrd) # Get candidates for each word
        uni_prob = [prob(correct_word) for correct_word in correct_word_list] # Get unigram probabilities for each candidate
        corrected_sentence[corrected_sentence.index(wrong_wrd)] = correct_word_list[uni_prob.index(max(uni_prob))] # Replace the mispelled word with candidate with highest uni prob
    return corrected_sentence

print(correct(["this","whitr","cat"]))

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

['this', 'white', 'cat']


## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [10]:
def accuracy(test):
    # Write your code here
    # Correct all original sentences in test dataset using model and compare the corrected sentence in test dataset
    correct_words_counter = 0
    total_words_counter = 0
    for sentence in test:
        total_words_counter += len(sentence['corrected'])
        correct_words_counter += sum([i == j for i, j in zip(sentence['corrected'], correct(sentence['original']))]) # Compare each word in both sentences
    
    return float(correct_words_counter/total_words_counter)

print("Accuracy of the algorithm is : " + str(100*accuracy(test)) + "%")

Accuracy of the algorithm is : 80.24118738404454%


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


To improve the accuracy of algorithm in task 3 and 4 we need to correct the words in test which are really mispelled and protect the words in test from being corrected if they are valid words but not present in train dataset.

List of implementations that can increase the accuracy of the algorithm or model designed in Task 3 and 4 :

1. As the train dataset is very small we need larger corpus of words to improve the accuracy of model. Use wordnet from nltk corpus by checking if a word is as valid word.

2. Stemming and lemmatization- Check whether the each word's stem is present in unique tokens from train and in wordnet because maybe word's stem or lemma might be present in unique words from train or wordnet words so its a valid word and need not be corrected.

3. Stopping digits from being corrected by the algorithm as for them the model will get the word with nearest distance but that will disimprove the accuracy.

4. Name Entity Recognition with NLTK - Named entity recognition (NER)is probably the first step towards information extraction that seeks to locate and classify named entities in text into pre-defined categories such as the names of persons, organizations, locations, expressions of times, quantities, monetary values, percentages, etc. Now in the given data there are many sentence which have names and entities which are not valid words for algorithm and they are corrected hence reducing the accuracy. This step significantly increases the accuracy of the algorithm . For example 2nd sentence in test dataset "NIGEL THRUSH page 48" is corrected to "WIGGLE THEN page 48" where "NIGEL" and "THRUSH" are name entities which need not be corrected.

5. Bigram Probabilty - Instead of using unigram probability use bigram probability which slighlty inceases the performance of the algorithm due to joint probability distribution of a sequence of words.

6. Pointwise Mutual Information(PMI) (Not implemented) - Use PMI instead of bigram probability as PMI of the given bigram is more then they are more likely to occur together in the wild. Hence, we can leverage this while assigning the conditional probability of a word based on its context (previous word). Likewise, the probability of the sentence can be calculated which is the multiplication of probabilities of each word in a sentence.\

7. Probabilistic modelling like Naïve Bayes	as Language	Model Based. (Not implemented) - Sentences are not large so not used.

# Implementation

In [11]:
# Create dictionary of bigram frequencies for each pair of words
training_sentence = []
for sent in train:
        training_sentence.append(" ".join(sent['corrected']))
        
training_sentence = " ".join(training_sentence)    
training_sentence = training_sentence.lower()
#Calculating Bigrams for given words.
tokens = nltk.wordpunct_tokenize(training_sentence)
bigrams=nltk.collocations.BigramCollocationFinder.from_words(tokens)
biag_freq = dict(bigrams.ngram_fd)

In [12]:
def names_entities(sentence):
    #Name Entity detection using nltk.pos_tag and nltk.ne_chunk
    ne_lst = []
    for word in nltk.ne_chunk(nltk.pos_tag(sentence)):
          if hasattr(word, 'label'): # Find a name entity like person name, location, organisation and append to list.
            ne_lst.append(' '.join(w[0] for w in word))
    
    if len(ne_lst) != 0:# Add all name entities tokens in list
        ne_lst = " ".join(ne_lst)
        ne_lst = ne_lst.split(" ")
        
    return ne_lst

In [13]:
# Implementation of bigram probability using bigram frequency dict created
def bigram(words):
    # If there is no such bigram return 0
    try:
        return float(biag_freq[tuple(map(str, words.lower().split(' ')))])/sum(biag_freq.values())
    except KeyError:
        return 0.0

In [14]:
def correct_modified(sentence):
    # Write your code here
    
    # This is to culculate unigram and bigram probabilities in correct function
    new_sentence = sentence.copy()
    stemmer = PorterStemmer()
    names = names_entities(new_sentence)
    for i in range(len(new_sentence)):
        if new_sentence[i].lower not in unique_words and new_sentence[i] not in names and not new_sentence[i].isdigit() and lemmatizer.lemmatize(new_sentence[i].lower()) not in unique_words and new_sentence[i] not in wn.words() and stemmer.stem(new_sentence[i]) not in wn.words() and lemmatizer.lemmatize(new_sentence[i].lower()) not in wn.words() and stemmer.stem(new_sentence[i]) not in unique_words:
            bigram_prob = []
            
            correct_word_list = get_candidates(new_sentence[i])
             
            for correct_word in correct_word_list:
                if i == 0:
                    bigram_prob.append(bigram("{} {}".format(correct_word, new_sentence[i+1])))
                else:
                    bigram_prob.append(bigram("{} {}".format(new_sentence[i-1],correct_word)))
            new_sentence[i] = correct_word_list[bigram_prob.index(max(bigram_prob))]
    return new_sentence

correct_modified(["this","whitr","cat"]) 

['this', 'white', 'cat']

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [15]:
def accuracy_new(test):
    # Write your code here

    correct_words_counter = 0
    total_words_counter = 0
    for sentence in test:
        total_words_counter+=len(sentence['corrected'])
        correct_words_counter += sum([i == j for i, j in zip(sentence['corrected'], correct_modified(sentence['original']))])
    
    return float(correct_words_counter/total_words_counter)

print("Accuracy of the modified algorithm is : " + str(100*accuracy_new(test)) + "%")

Accuracy of the modified algorithm is : 91.1873840445269%


Result : After modifying the algorithm in task 3 and 4 using the implementations discussed in task 6 the accuracy of the model
is increased by aprrox 11 i.e. from 80 % to 91%.